# LOADING LIBS

In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.read_csv('/kaggle/input/movie-dataset-v3/movies_genres.csv', delimiter='\t')

In [34]:
df.shape

(117352, 30)

In [35]:
df.head()

,title,plot,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the...",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [19]:
from concurrent.futures import ThreadPoolExecutor
from langdetect import detect

def detect_lang(plot):
    return detect(plot)

with ThreadPoolExecutor() as executor:
    lang_list = list(tqdm(executor.map(detect_lang, df['plot']), total=df.shape[0]))

df['plot_lang'] = lang_list
df = df[df.plot_lang.isin(['en'])]

100%|██████████| 117352/117352 [13:59<00:00, 139.80it/s]


In [20]:
import pickle

with open('df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [36]:
import pickle 

with open('/kaggle/input/movie-dataset-v3/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [37]:
df.head()

,title,plot,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,plot_lang
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,en
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the...",0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,en
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,en
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,en
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,en


In [38]:
df['genre'] = df.apply(lambda row: [col.split('_')[-1] for col in df.columns[2:-1] if row[col] == 1], axis=1)
df = df.iloc[:, :2].join(df.iloc[:, -1])

In [39]:
df.head()

,title,plot,genre
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...,[Comedy]
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the...",[Reality-TV]
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...,[Reality-TV]
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...,[Reality-TV]
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...,[Reality-TV]


In [40]:
df['plot_genre'] = df['genre'].apply(lambda x: ' '.join(x)) + ' ' + df['plot']

df = df.iloc[:, :1].join(df.iloc[:, -1])

In [41]:
df.head()

,title,plot_genre
0,"""#7DaysLater"" (2013)",Comedy #7dayslater is an interactive comedy s...
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}",Reality-TV With just one week left in the wor...
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",Reality-TV All of the women start making stri...
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",Reality-TV All five of these women are indepe...
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Reality-TV Despite having gone through a life...


In [71]:
import pickle

with open('df_preprocessed.pkl', 'wb') as f:
    pickle.dump(df, f)

In [42]:
with open('/kaggle/input/movie-dataset-v3/df_preprocessed.pkl', 'rb') as f:
    df = pickle.load(f)

In [43]:
import nltk
from nltk.corpus import stopwords
import spacy
from multiprocessing import Pool

nltk.download('stopwords')

nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
def preprocess_text(text):
    doc = nlp(text.lower())
    words = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return ' '.join(words)

tqdm.pandas()
df['plot_genre'] = df['plot_genre'].progress_apply(preprocess_text)

In [99]:
import pickle

with open('df_preprocessed_2.pkl', 'wb') as f:
    pickle.dump(df, f)

In [44]:
import pickle

with open('/kaggle/input/movie-dataset-v3/df_preprocessed_2.pkl', 'rb') as f:
    df = pickle.load(f)
    
df_output = pd.read_csv('/kaggle/input/movie-dataset-v3/movies_genres.csv', delimiter='\t')
df_output = df_output.iloc[:,:2]

df_output.head()

,title,plot
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the..."
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...


In [45]:
import re

df['title'] = df['title'].apply(lambda x: re.findall(r'"([^"]*)"', x)[0])
df['plot'] = df_output['plot']
df.head()

,title,plot_genre,plot
0,#7DaysLater,comedy interactive comedy series feature ensem...,#7dayslater is an interactive comedy series f...
1,#BlackLove,reality tv week leave workshop woman consider ...,"With just one week left in the workshops, the..."
2,#BlackLove,reality tv woman start make stride find versio...,All of the women start making strides towards...
3,#BlackLove,reality tv woman independent strong willed fac...,All five of these women are independent and s...
4,#BlackLove,reality tv despite having go life change proce...,Despite having gone through a life changing p...


In [6]:
from tqdm import tqdm
from collections import defaultdict

# Create inverted index
inverted_index = defaultdict(list)
for doc_id, doc_text in tqdm(df['plot_genre'].items(), total=len(df)):
    for word in doc_text.split():
        if word not in inverted_index:
            inverted_index[word] = []
        if doc_id not in inverted_index[word]:
            inverted_index[word].append(doc_id)

100%|██████████| 117193/117193 [04:29<00:00, 435.62it/s]


In [12]:
from tqdm import tqdm
from collections import defaultdict

inverted_index = defaultdict(set)

for doc_id, doc_text in tqdm(df['plot_genre'].items(), total=len(df)):
    words = set(doc_text.split())
    for word in words:
        inverted_index[word].add(doc_id)

100%|██████████| 117193/117193 [00:04<00:00, 27104.50it/s]


In [7]:
import pickle

with open('inverted_index.pkl', 'wb') as f:
    pickle.dump(inverted_index, f)

In [46]:
import pickle
import pandas as pd
from tqdm import tqdm

with open('/kaggle/input/movie-dataset-v3/inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

In [47]:
!pip install rank_bm25

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import numpy as np

# Define the query
query = 'spiderman'
query = preprocess_text(query)

# Create a list of preprocessed document texts
docs = list(df['plot_genre'])

# Use the inverted index to generate a vocabulary for tf-idf vectorization
vocab = list(inverted_index.keys())

# Define the vectorizers
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocab, norm=None)
bm25_vectorizer = BM25Okapi(docs)

# Vectorize the documents using tf-idf
tfidf_matrix = tfidf_vectorizer.fit_transform(docs)

# Vectorize the query using tf-idf
query_vec_tfidf = tfidf_vectorizer.transform([query])

# Get the tf-idf scores for the query
tfidf_scores = np.array(tfidf_matrix.dot(query_vec_tfidf.T).todense()).flatten()

# Get the BM25 scores for the query
bm25_scores = bm25_vectorizer.get_scores(query.split())

# Combine the scores using a weighted sum (you can experiment with different weights)
combined_scores = 0.5 * tfidf_scores + 0.5 * bm25_scores

# Get the indices of the top-k documents based on the combined scores
k = 10
top_k_indices = np.argsort(combined_scores)[::-1][:k]

# Get the titles of the top-k documents
for i in range(10):
    doc_id = top_k_indices[i]
    print("Rank:", i+1, "Title:",df.iloc[doc_id]['title'], "Doc ID:", doc_id, "Score:", combined_scores[doc_id])
    print(df.iloc[doc_id]['plot'])
    print("****************************************************")

Rank: 1 Title: Supaidâman Doc ID: 84522 Score: 132.92067465432717
 The Iron Cross Army suspects Spiderman's secret identity and uses it to kidnap his closest friends and family. To confirm their suspicions, the Iron Cross Army tries to force Takuya to change into Spiderman while in public. Now,without being able to change into Spiderman, Takuya must try to save his friends and family before it's too late.
****************************************************
Rank: 2 Title: The Amazing Spider-Man Doc ID: 88130 Score: 88.61378310288478
 Peter goes and takes photos of a scientist who has perfected the art of cloning, cloning a frog. At the same time, a member of a scientific group who gives scientific awards is killed. And the police get a of description of the person who did it and it's the scientist. But how could have been there when he was at the demonstration. Later another attempt is made on another member of the group but Spiderman saves her, but the woman refuses to implicate the s

In [26]:
import math

# compute the document frequency (DF) for each term in the corpus
DF = {}
for term in inverted_index:
    DF[term] = len(inverted_index[term])

documents = df['plot_genre']
# compute the inverse document frequency (IDF) for each term in the corpus
N = len(documents)  # number of documents in the corpus
idf = {}
for term in inverted_index:
    idf[term] = math.log(N / DF[term])

# compute the TF-IDF weight for each term in each document
tfidf = {}
for doc_id, doc_text in tqdm(documents.items(), total=len(documents)):
    tfidf[doc_id] = {}
    for term in doc_text.split():
        if term in inverted_index:
            tf = list(inverted_index[term]).count(doc_id)
            tfidf[doc_id][term] = tf * idf[term]

 26%|██▌       | 30358/117193 [05:16<15:05, 95.89it/s] 


KeyboardInterrupt: 